<a href="https://colab.research.google.com/github/justinfmccarty/epwmorph/blob/main/morphing_routines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
# !pip install metpy
!pip install meteocalc

  Created wheel for meteocalc: filename=meteocalc-1.1.0-cp36-none-any.whl size=8196 sha256=253edaca69913465e7872a70366df4cc313c4f909f9afe6d4da70081f1a7926f
  Stored in directory: /root/.cache/pip/wheels/9e/34/13/83d36ecc28837e3c2a5b696542e697538e7c1025382f4ded55
Successfully built meteocalc


In [81]:
import pandas as pd
# import metpy.calc as mpcalc
# from metpy.units import units
import numpy as np
import datetime as dt
import math
from meteocalc import dew_point as calcdewpt

In [4]:
def epw_to_dataframe(weather_path):
    epw_labels = ['year', 'month', 'day', 'hour', 'minute', 'datasource', 'drybulb_C', 'dewpoint_C', 'relhum_percent',
                  'atmos_Pa', 'exthorrad_Whm2', 'extdirrad_Whm2', 'horirsky_Whm2', 'glohorrad_Whm2',
                  'dirnorrad_Whm2', 'difhorrad_Whm2', 'glohorillum_lux', 'dirnorillum_lux', 'difhorillum_lux',
                  'zenlum_lux', 'winddir_deg', 'windspd_ms', 'totskycvr_tenths', 'opaqskycvr_tenths', 'visibility_km',
                  'ceiling_hgt_m', 'presweathobs', 'presweathcodes', 'precip_wtr_mm', 'aerosol_opt_thousandths',
                  'snowdepth_cm', 'days_last_snow', 'Albedo', 'liq_precip_depth_mm', 'liq_precip_rate_Hour']

    df = pd.DataFrame(pd.read_csv(weather_path, skiprows=8, header=None, names=epw_labels).drop('datasource', axis=1))
    return df



In [5]:
df = epw_to_dataframe('/content/CAN_BC_Vancouver.Intl.AP.718920_CWEC2016.epw')
# returns datetime objects
df.index = pd.to_datetime(df.apply(lambda row: dt.datetime(2016, 
                                                           int(row.month), 
                                                           int(row.day), 
                                                           int(row.hour)-1,
                                                           int(row.minute)), axis=1))
df.head()

,year,month,day,hour,minute,drybulb_C,dewpoint_C,relhum_percent,atmos_Pa,exthorrad_Whm2,extdirrad_Whm2,horirsky_Whm2,glohorrad_Whm2,dirnorrad_Whm2,difhorrad_Whm2,glohorillum_lux,dirnorillum_lux,difhorillum_lux,zenlum_lux,winddir_deg,windspd_ms,totskycvr_tenths,opaqskycvr_tenths,visibility_km,ceiling_hgt_m,presweathobs,presweathcodes,precip_wtr_mm,aerosol_opt_thousandths,snowdepth_cm,days_last_snow,Albedo,liq_precip_depth_mm,liq_precip_rate_Hour
2016-01-01 00:00:00,2011,1,1,1,0,-1.8,-4.5,80,102160,0,0,238,0,0,0,0,0,0,0,0,0.0,0,0,48.3,77770,0,999999990,0,0.085,0,88,0.2,0.0,0.0
2016-01-01 01:00:00,2011,1,1,2,0,-3.0,-5.3,82,102010,0,0,233,0,0,0,0,0,0,0,0,0.0,0,0,48.3,77770,0,999999990,0,0.085,0,88,0.2,0.0,0.0
2016-01-01 02:00:00,2011,1,1,3,0,-4.0,-6.0,84,101920,0,0,229,0,0,0,0,0,0,0,0,0.0,0,0,48.3,77770,0,999999990,0,0.085,0,88,0.2,0.0,0.0
2016-01-01 03:00:00,2011,1,1,4,0,-4.7,-6.4,86,101880,0,0,226,0,0,0,0,0,0,0,0,0.0,0,0,48.3,77770,0,999999990,0,0.085,0,88,0.2,0.0,0.0
2016-01-01 04:00:00,2011,1,1,5,0,-5.2,-6.7,88,101880,0,0,224,0,0,0,0,0,0,0,0,0.0,0,0,48.3,77770,0,999999990,0,0.085,0,88,0.2,0.0,0.0


Dry Bulb

In [45]:


fut_tas = pd.Series([-5,3,5,9,13,15,17,19,24,15,11,-1])
fut_tmax = pd.Series([3,13,15,19,22,25,27,29,33,25,19,10])
fut_tmin = pd.Series([-6,1,3,6,9,10,11,10,18,13,8,-4])
hist_tas = pd.Series([-7,0,2,6,10,12,15,18,21,14,6,-3])
hist_tmax = pd.Series([2,11,10,14,20,22,25,27,26,18,13,5])
hist_tmin = pd.Series([-12,-5,-3,0,1,5,6,10,12,11,0,-10])


def means(series):
  max = series.resample('D').max().resample('M').mean().reset_index(drop=True)
  min = series.resample('D').min().resample('M').mean().reset_index(drop=True)
  mean = series.resample('D').mean().resample('M').mean().reset_index(drop=True)
  return max, min, mean

def change(fut_tas,hist_tas,fut_tmax,hist_tmax,fut_tmin,hist_tmin):
  tas = fut_tas-hist_tas
  tmax = fut_tmax-hist_tmax
  tmin = fut_tmin-hist_tmin
  return tas, tmax, tmin

def morph_dbt(df, fut_tas,hist_tas,fut_tmax,hist_tmax,fut_tmin,hist_tmin):
  months = list(range(1, 12 + 1, 1))
  dbt_max_mean, dbt_min_mean, dbt_mean = means(df['drybulb_C'])
  tas_change, tmax_change, tmin_change = change(fut_tas, hist_tas,
                                                fut_tmax, hist_tmax,
                                                fut_tmin, hist_tmin)
  tas_change = dict(zip(months, tas_change)) 
  dbt_scale = dict(zip(months, (tmax_change - tmin_change) / (dbt_max_mean - dbt_min_mean)))
  dbt_mean = dict(zip(months, dbt_mean))
  return df.apply(lambda x: x['drybulb_C'] + tas_change[x['month']] + dbt_scale[x['month']] * (x['drybulb_C'] - dbt_mean[x['month']]),axis=1)

new_dbt = morph_dbt(df, fut_tas,hist_tas,fut_tmax,hist_tmax,fut_tmin,hist_tmin).rename("drybulb_C")


Relative Humidity

In [92]:
fut_rh = pd.Series([35,45,55,55,78,77,95,65,70,60,80,70])
hist_rh = pd.Series([40,60,54,50,76,70,88,61,72,64,85,75])

def morph_relhum(df, fut_relhum, hist_relhum):
    # requires fut_ and hist_ inputs to be monthly climatologies
    months = list(range(1,12+1,1))
    relhum_change = dict(zip(months, fut_relhum - hist_relhum))
    return df.apply(lambda x: x['relhum_percent'] + relhum_change[x['month']],axis=1)

new_rh = np.clip(morph_relhum(df, fut_rh, hist_rh).rename("relhum_percent"),0,100).astype(int)

Pressure

In [47]:
fut_pr = pd.Series([101860,101865,101880,101880,102160,102230,101890,101950,102110,101890,101980,102380])
hist_pr = pd.Series([101890,101855,101870,101830,102100,101880,101880,101890,102160,101880,101865,102180])

def morph_pr(df, fut_pr, hist_pr):
    # requires fut_ and hist_ inputs to be monthly climatologies
    months = list(range(1,12+1,1))
    pr_change = dict(zip(months, fut_pr - hist_pr))
    return df.apply(lambda x: x['atmos_Pa'] + pr_change[x['month']],axis=1)

new_pr = morph_pr(df, fut_pr, hist_pr).rename("atmos_Pa")


Dew Point

In [60]:
def calc_sat_pr(pressure, dbt, rel_hum):
    # pressure units in in Pa
    # pressure units out in Pa
    # temp units in C
    mixing = mpcalc.mixing_ratio_from_relative_humidity(pressure*units.Pa,
                                                        dbt*units.degC,
                                                        rel_hum*units.percent)
    return mpcalc.vapor_pressure(pressure*units.Pa, mixing)

def calc_partial_water_pr(pressure, dbt, rel_hum):
    # pressure units in in Pa
    # pressure units out in kPa
    # temp units in C
    # rel hum in %
    return (rel_hum * calc_sat_pr(pressure, dbt, rel_hum)) / 1000

def morph_dewpt(epw_dewpt, epw_pressure, epw_dbt, epw_rh):
    pw = calc_partial_water_pr(epw_pressure, epw_dbt, epw_rh)
    if epw_dewpt>=0:
        epw_dewpt_fut = 6.54 + 14.526 + np.log(pw) + 0.7389 * np.log(pw)**2 + 0.09486 * np.log(pw)**3 + 0.4569 * pw**0.1984
    else:
        epw_dewpt_fut = 6.09 + 12.608 * np.log(pw) + 0.4959 * np.log(pw)**2
    return epw_dewpt_fut

# df.apply(lambda x: morph_dewpt(x['dewpoint_C'], 
#                                x['atmos_Pa'], 
#                                x['drybulb_C'], 
#                                x['relhum_percent']), axis=1)

In [107]:
def calcdew_above(pw):
  return 6.54 + 14.526 + np.log(pw) + 0.7389 * np.log(pw)**2 + 0.09486 * np.log(pw)**3 + 0.4569 * pw**0.1984

def calcdew_sub(pw):
  return 6.09 + 12.608 * np.log(pw) + 0.4959 * np.log(pw)**2

In [96]:
df_new = pd.concat([new_dbt,new_rh,new_pr],axis=1)

In [101]:
# calc_partial_water_pr(df_new['atmos_Pa'][2], df_new['drybulb_C'][2], df_new['relhum_percent'][2])
df_new['dewpt'] = df_new.apply(lambda x: calcdewpt(x['drybulb_C'], x['relhum_percent']),axis=1)
df_new['vapor'] = df_new.apply(lambda x: calc_sat_pr(x['atmos_Pa'], x['drybulb_C'], x['relhum_percent']),axis=1)
df_new['partial'] = df_new.apply(lambda x: calc_partial_water_pr(x['atmos_Pa'], x['drybulb_C'], x['relhum_percent']),axis=1)

In [ ]:
df_new['old_dp'] = df['dewpoint_C']


In [ ]:
df_new['dewpoint_met'] = np.where(df_new['old_dp']>=0,
                                  df_new['partial'].apply(lambda x: calcdew_above(x.magnitude)),
                                  df_new['partial'].apply(lambda x: calcdew_sub(x.magnitude)))
df_new.iloc[3000:3024]

In [119]:
df_old = df[['drybulb_C','atmos_Pa','relhum_percent','dewpoint_C']].copy()
df_old['dewpt2'] = df_old.apply(lambda x: calcdewpt(x['drybulb_C'], x['relhum_percent']),axis=1)
df_old['vapor'] = df_old.apply(lambda x: calc_sat_pr(x['atmos_Pa'], x['drybulb_C'], x['relhum_percent']),axis=1)
df_old['partial'] = ((df_old['vapor']/1000)*df_old['relhum_percent'])

df_old['dewpoint_met'] = np.where(df_old['dewpoint_C']>=0,
                                  df_old['partial'].apply(lambda x: calcdew_above(x.magnitude)),
                                  df_old['partial'].apply(lambda x: calcdew_sub(x.magnitude)))

df_old

,drybulb_C,atmos_Pa,relhum_percent,dewpoint_C,dewpt2,vapor,partial,dewpoint_met
2016-01-01 00:00:00,-1.8,102160,80,-4.5,-4.7882876840656365,429.1203304171651 pascal,34.32962643337321 pascal,56.872413
2016-01-01 01:00:00,-3.0,102010,82,-5.3,-5.635368331144875,402.39050228841666 pascal,32.996021187650165 pascal,56.234687
2016-01-01 02:00:00,-4.0,101920,84,-6.0,-6.299535616520955,382.4777867306467 pascal,32.12813408537432 pascal,55.806543
2016-01-01 03:00:00,-4.7,101880,86,-6.4,-6.680330798618282,371.45150611128804 pascal,31.94482952557077 pascal,55.714729
2016-01-01 04:00:00,-5.2,101880,88,-6.7,-6.873666241148814,365.95096966803254 pascal,32.20368533078686 pascal,55.844242
...,...,...,...,...,...,...,...,...
2016-12-31 19:00:00,2.4,102650,77,-1.1,-1.2125057626983655,560.1185097114892 pascal,43.12912524778467 pascal,60.575517
2016-12-31 20:00:00,2.1,102630,77,-1.4,-1.5035950789659067,548.2552783014881 pascal,42.21565642921458 pascal,60.225919
2016-12-31 21:00:00,1.7,102580,76,-1.9,-2.0685564404057106,525.901120152879 pascal,39.968485131618806 pascal,59.334695
2016-12-31 22:00:00,0.8,102480,77,-2.5,-2.765107366019474,499.3849640971867 pascal,38.452642235483374 pascal,58.706536
